In [1]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.ml.recommendation import ALS

spark = (pyspark.sql.SparkSession.builder
    .master("local")
    .getOrCreate())

For this example, first just a quick visual of a sparse user-item matrix, commonly what is used in recommendation systems.  This is a completely fictious one to recommend cheese to Muppets. Use whatever two things come to mind first! As you can see all users have rated something, but we have many unknowns.

In [2]:
muppet_cheese = [
{'muppet': 'Fozzie','gouda': 5, 'cheddar': 4,
 'brie': '-', 'swiss': 1, 'roquefort': '-'},
{'muppet': 'Kermit the Frog', 'gouda': '-', 'cheddar': 3,
 'brie': 4, 'swiss': 5, 'roquefort': '-'},
{'muppet': 'Rowlf the Dog', 'gouda': 2, 'cheddar': 4,
 'brie': 3, 'swiss': '-', 'roquefort': '-'},
{'muppet': 'Gonzo', 'gouda': '-', 'cheddar': '-',
 'brie': '-', 'swiss': 3, 'roquefort': 4},
{'muppet': 'Miss Piggy', 'gouda': 5, 'cheddar': 1,
 'brie': 3, 'swiss': '-', 'roquefort': 5}
        ]

mc = pd.DataFrame(muppet_cheese)

In [3]:
mc[['muppet', 'brie', 'cheddar', 'gouda', 'roquefort', 'swiss']]

,muppet,brie,cheddar,gouda,roquefort,swiss
0,Fozzie,-,4,5,-,1
1,Kermit the Frog,4,3,-,-,5
2,Rowlf the Dog,3,4,2,-,-
3,Gonzo,-,-,-,4,3
4,Miss Piggy,3,1,5,5,-


For creating the ALS model in Spark, data would typically more resemble the following format, each row a record of a user-item interaction.  For Spark, you also need to make sure the user and item are numeric, they cannot be text. 

In [4]:
muppet_cheese_als = [
    {'muppet': 1,'item':101, 'score': 5},
    {'muppet': 1,'item': 201, 'score': 4},
    {'muppet': 1,'item':401, 'score': 1},
    {'muppet': 2,'item': 201, 'score': 3},
    {'muppet': 2,'item': 301, 'score': 4},
    {'muppet': 2,'item':401, 'score': 5},
    {'muppet': 3,'item':101, 'score': 2},
    {'muppet': 3,'item': 201, 'score': 4},
    {'muppet': 3,'item': 301, 'score': 3},
    {'muppet': 4,'item':401, 'score': 3},
    {'muppet': 4,'item': 501, 'score': 4},
    {'muppet': 5,'item':101, 'score': 5},
    {'muppet': 5,'item': 201, 'score': 1},
    {'muppet': 5,'item': 301, 'score': 3},
    {'muppet': 5,'item': 501, 'score': 5},
        ]
muppet_cheese_als_df = pd.DataFrame(muppet_cheese_als)

In [5]:
muppets = spark.createDataFrame(muppet_cheese_als_df)

Typically, you would create train-test split to validate the results, but this is a small silly Muppet example, so we throw it all into our model:

In [6]:
als = ALS(rank=10, regParam=.1, maxIter=20,
          userCol='muppet', itemCol='item', 
          ratingCol='score', nonnegative=True)

als_model = als.fit(muppets)

Separating the items and user features to show what they look like:

In [7]:
items = als_model.itemFactors.toPandas()
users = als_model.userFactors.toPandas()

In [8]:
# Item Features
for i in range(10):
    items[str(i + 1)] = items['features'].apply(lambda x: x[i])
items['cheese'] = ['gouda', 'cheddar', 'brie', 'swiss', 'roquefort']
#users['muppet'] = ['Fozzie', 'Kermit the Frog', 'Rowlf the Dog', 'Gonzo', 'Miss Piggy']
items[['cheese', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']]

,cheese,1,2,3,4,5,6,7,8,9,10
0,gouda,0.0,0.651199,1.812686,1.055743,0.000000,0.805005,0.164712,0.000000,0.000000,1.035070
1,cheddar,0.0,0.000000,0.576444,1.584922,0.460479,0.113503,0.449816,1.079774,0.095524,0.000000
2,brie,0.0,0.673650,0.819783,0.758670,0.434579,0.387765,0.248254,1.014423,0.089965,0.755413
3,swiss,0.0,1.009534,0.220818,0.000000,0.649082,0.235954,0.131635,1.506025,0.134003,0.941965
4,roquefort,0.0,1.124071,1.402186,0.208343,0.150048,0.758195,0.032496,0.347897,0.030967,1.405531


In [9]:
# User Features
for i in range(10):
    users[str(i + 1)] = users['features'].apply(lambda x: x[i])
users['muppet'] = ['Fozzie', 'Kermit the Frog', 'Rowlf the Dog', 'Gonzo', 'Miss Piggy']
users[['muppet', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']]

,muppet,1,2,3,4,5,6,7,8,9,10
0,Fozzie,0.0,0.033114,1.321696,1.624912,0.085373,0.448752,0.341840,0.205471,0.017924,0.271322
1,Kermit the Frog,0.0,0.929531,0.402171,0.371168,0.670217,0.272178,0.217702,1.557829,0.138478,0.887746
2,Rowlf the Dog,0.0,0.131508,0.310402,1.186012,0.515610,0.047636,0.384975,1.207027,0.106879,0.070795
3,Gonzo,0.0,0.813947,0.768313,0.106355,0.230989,0.443168,0.047899,0.535821,0.047682,0.941555
4,Miss Piggy,0.0,0.831885,1.297370,0.202528,0.000525,0.674584,0.000000,0.000000,0.000009,1.120296


In [10]:
# Combining User & Item features:
for idx, cheese in enumerate(items['cheese'].tolist()):
    users[cheese] = users['features'].apply(lambda x: np.dot(x, items['features'][idx]))
users['muppet'] = ['Fozzie', 'Kermit the Frog', 'Rowlf the Dog', 'Gonzo', 'Miss Piggy']
users[['muppet', 'brie', 'cheddar', 'gouda', 'swiss', 'roquefort']]

,muppet,brie,cheddar,gouda,swiss,roquefort
0,Fozzie,3.049565,3.804828,4.831264,1.099002,3.046574
1,Kermit the Frog,3.951685,2.952871,2.900019,4.756014,3.794116
2,Rowlf the Dog,2.868492,3.788194,2.075458,2.496714,1.478892
3,Gonzo,2.802075,1.372787,3.374253,2.952427,3.897904
4,Miss Piggy,2.885708,1.145661,4.809893,2.341090,4.882601


We can now compare this matrix with the original to see what some of our best recommendations are:

In [11]:
mc[['muppet', 'brie', 'cheddar', 'gouda', 'swiss', 'roquefort']]

,muppet,brie,cheddar,gouda,swiss,roquefort
0,Fozzie,-,4,5,1,-
1,Kermit the Frog,4,3,-,5,-
2,Rowlf the Dog,3,4,2,-,-
3,Gonzo,-,-,-,3,4
4,Miss Piggy,3,1,5,-,5


Next, we can walk through using these features to get recommendations for a new user without re-running the model. First, we get the item features for the items we have ratings for (in this scenario, gouda & cheddar):

In [12]:
select_cheese = items.iloc[0:2, 2:12].to_numpy()

In [13]:
select_cheese.shape

(2, 10)

The legendary Muppet Dr. Teeth has rated both of these cheeses very highly. We put those score in an array of the corresponding amount of rows:

In [14]:
dr_teeth = np.array([[5],[5]])
dr_teeth.shape

(2, 1)

We then do the linear algebra to take our known ratings and know item features to return a full set of user features:

In [15]:
dr_teeth_features = np.linalg.lstsq(select_cheese, dr_teeth, rcond=None)
dr_teeth_features = dr_teeth_features[0].reshape((10,))
dr_teeth_features

array([0.        , 0.24345039, 1.18610604, 1.79262076, 0.40615101,
       0.40106219, 0.45832383, 0.95238176, 0.08425429, 0.38696028])

Tada! Now we can use these features to get a full set of recommendations by matrix multiplication with all the item features:

In [16]:
dr_teeth_scores = [np.dot(dr_teeth_features, row) for row in items.iloc[:, 2:12].to_numpy()]
for row in zip(items['cheese'], dr_teeth_scores):
    print(row)

('gouda', 5.000000000000002)
('cheddar', 4.999999999999999)
('brie', 4.208173968183433)
('swiss', 2.736376971886836)
('roquefort', 3.5680200534574915)
